## Tf-Explain technique of explaining neural networks
From https://github.com/sicara/tf-explain

Before you start remember to create virtual environment. Type in your shell: 

```virtualenv venv -p python3.6```

Create new kernel using your virtualenv (https://anbasile.github.io/posts/2017-06-25-jupyter-venv/) and turn it on in the notebook **(Top bar > Kernel > Change kernel > ...).**

#### Install all dependencies

In [23]:
pip install tf-explain
pip install tensorflow==2.1.0
pip install pillow
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


#### Import all libraries and classes

In [165]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import sys
from glob import glob
from PIL import Image
sys.modules['Image'] = Image

import tensorflow as tf
import numpy as np
import keras
import cv2 as cv
from tf_explain.core import *

Be sure that your keras backend is switch to 'tensorflow' and your image data format is 'channels_last'

In [68]:
tf.keras.backend.set_image_data_format('channels_last')

#### Loading pretrained model

In [153]:
model_vgg16 = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)

In [209]:
model_inception = tf.keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet')

#### Create list of image paths and class indexes of images from ImageNet
You have to organise your images like: *./images/xyz/* where xyz is class index from ImageNet (e.g. ./images/96/)

In [247]:
classes_list = glob('./images/*')

In [252]:
data_list = []

for class_path in classes_list:
    class_index = class_path.split('/')[-1]
    path_for_glob = class_path + '/*.jpg'
    images_paths = glob(path_for_glob)
    
    for image in images_paths:
        
        print(image)
        data_list.append((image,int(class_index)))

./images/96/toucan.jpg
./images/96/toucanet.jpg
./images/96/toucanet_2.jpg
./images/71/real_scorpion.jpg
./images/71/scorpion.jpg


#### Create diffrent types of explainers

In [253]:
# Start explainer
explainer_grad_cam = GradCAM()
explainer_vanilla = VanillaGradients()
explainer_smooth = SmoothGrad()
explainer_occlusion = OcclusionSensitivity()
explainer_int_grad = IntegratedGradients()

#### Load images from path and start explaining with all five techniques and save the result images
Make sure, you've chosen right model (inception or vgg16)

In [254]:
for data_item in data_list:
    
    image_path = data_item[0]
#     print(image_path)
    image_name = image_path.split('/')[-1][:-4]
#     print(image_name)

    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img = tf.keras.preprocessing.image.img_to_array(img)
    
    data = ([img], None)
    
    grid_cam = explainer_grad_cam.explain(data, model_inception, class_index=data_item[1], colormap=cv.COLORMAP_JET)
    explainer_grad_cam.save(grid_cam, "./images/" + str(data_item[1])+ "/results/", image_name + "-grad_cam.png")
        
#     grid_occlusion = explainer_occlusion.explain(data, model_inception, class_index=data_item[1], colormap=cv.COLORMAP_JET)
#     explainer_occlusion.save(grid_occlusion, "./images/" + str(data_item[1])+ "/", image_name + "-occlusion_sensitivity.png")

    print("-----------------DONE-------------------")


    
#     grid_vanilla = explainer_vanilla.explain(data_np_array, model, scorpion_class_index)
#     explainer_vanilla.save(grid_vanilla, ".", "vanilla_gradients.png")

#     grid_smooth = explainer_smooth.explain(data, model, scorpion_class_index, 20, 1.0)
#     explainer_smooth.save(grid_smooth, ".", "smoothgrad.png")

#     grid_grad = explainer_int_grad.explain(data, model, scorpion_class_index, n_steps=15)
#     explainer_int_grad.save(grid_grad, ".", "integrated_gradients.png")
    





./images/96/toucan.jpg
toucan
./images/96/toucanet.jpg
toucanet
./images/96/toucanet_2.jpg
toucanet_2
./images/71/real_scorpion.jpg
real_scorpion
./images/71/scorpion.jpg
scorpion
